## Scraping (bloqué)

In [3]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import json

In [18]:
cookies = {"earchives_vaucluse_fr_bach_cookie":"%7B%22map%22%3Afalse%2C%22daterange%22%3Afalse%2C%22piwik_ignore%22%3A%221%22%2C%22social_ignore%22%3A%221%22%7D", "PHPSESSID":"d86cef0537293c70af58667a920c14ef"}

In [19]:
headers= {"User-Agent": "Mozilla/5.0"}
response = requests.get("https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-11", headers=headers, cookies=cookies)
print(response.text)

You got banned permanently from this server.


## Scraping Selecium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By